### PyImageJ Testing 
Testing PyImageJ's capabilities to do particle image processing

In [1]:
import imagej
from imagej import Mode

Initializing ImageJ 

In [2]:
# initalize imagej
ij = imagej.init('net.imagej:imagej+net.imagej:imagej-legacy', mode=Mode.HEADLESS)

Opening Image

In [3]:
# macro command 
input_image = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Particle Images/Ti64_Lot232-EZ2316_1Use_10X_Scale.png'
output_image = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Processed Testing Output/processed_img.png'
results_summary = '/Users/sarah/Documents/areospace research/material recycling project/PyImageJ-Particle-Analysis/Processed Testing Output/results_summary.csv'

macro_cmd = f"""
// open image file 
open("{input_image}");

// convert image to 8-bit image type
run("8-bit");

// set threshold for image and invert
setThreshold(70, 255);
run("Invert");

// convert to a binary mask 
run("Convert to Mask");

// save preprocessed image
saveAs("png", "{output_image}");

// implement scale 
 run("Set Scale...", "distance=180 known=100 unit=um"); 

// set measurements
run("Set Measurements...", "area mean standard min centroid perimeter bounding box fit shape feret's integrated density median skewness kurtosis area_fraction stack position")

// analyze particles 
run("Analyze Particles...","size=0-Infinity exclude");

// save analyzed particle results to summary csv file
saveAs("Results", "{results_summary}");
"""

ij.py.run_macro(macro_cmd)

Operating in headless mode - the original ImageJ will have limited functionality.


<java object 'org.scijava.script.ScriptModule'>